# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17464789/17464789 [==============================] - 5s 0us/step
25000 train sequences
25000 test sequences


In [6]:
# This pads (or truncates) the sequences so that they are of the maximum length
from keras.utils import pad_sequences
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [7]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [12]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='tanh',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          1000000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5)                 280       
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [14]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 11ms/step - loss: 0.6634 - accuracy: 0.6106 - val_loss: 0.6306 - val_accuracy: 0.6570
Epoch 2/10
782/782 [==============================] - 9s 11ms/step - loss: 0.6012 - accuracy: 0.6913 - val_loss: 0.5957 - val_accuracy: 0.6932
Epoch 3/10
782/782 [==============================] - 8s 11ms/step - loss: 0.5589 - accuracy: 0.7338 - val_loss: 0.5697 - val_accuracy: 0.7118
Epoch 4/10
782/782 [==============================] - 8s 11ms/step - loss: 0.5202 - accuracy: 0.7634 - val_loss: 0.5407 - val_accuracy: 0.7288
Epoch 5/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4767 - accuracy: 0.7903 - val_loss: 0.5028 - val_accuracy: 0.7569
Epoch 6/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4411 - accuracy: 0.8079 - val_loss: 0.4871 - val_accuracy: 0.7652
Epoch 7/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4168 - accuracy: 0.8220 - val_loss: 0.4767 - val_accuracy: 0.77

In [16]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 2ms/step - loss: 0.4569 - accuracy: 0.7872
Test score: 0.4568624496459961
Test accuracy: 0.7871599793434143


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [18]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [19]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [20]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [21]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 15s 18ms/step - loss: 0.6028 - accuracy: 0.6746 - val_loss: 0.5434 - val_accuracy: 0.7276
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.4595 - accuracy: 0.7830 - val_loss: 0.4565 - val_accuracy: 0.7856
Epoch 3/10
782/782 [==============================] - 14s 17ms/step - loss: 0.3938 - accuracy: 0.8252 - val_loss: 0.4247 - val_accuracy: 0.8062
Epoch 4/10
782/782 [==============================] - 14s 17ms/step - loss: 0.3510 - accuracy: 0.8482 - val_loss: 0.4043 - val_accuracy: 0.8156
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3222 - accuracy: 0.8636 - val_loss: 0.3947 - val_accuracy: 0.8217
Epoch 6/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3015 - accuracy: 0.8746 - val_loss: 0.3775 - val_accuracy: 0.8308
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2856 - accuracy: 0.8826 - val_loss: 0.3802 - val_accuracy:

In [23]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [24]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [25]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [26]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6695 - accuracy: 0.5596 - val_loss: 0.6346 - val_accuracy: 0.7271
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5983 - accuracy: 0.7278 - val_loss: 0.5899 - val_accuracy: 0.7372
Epoch 3/10
782/782 [==============================] - 11s 13ms/step - loss: 0.5649 - accuracy: 0.7774 - val_loss: 0.5690 - val_accuracy: 0.7781
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5435 - accuracy: 0.7991 - val_loss: 0.5588 - val_accuracy: 0.7894
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5271 - accuracy: 0.8161 - val_loss: 0.5469 - val_accuracy: 0.7997
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5123 - accuracy: 0.8243 - val_loss: 0.5379 - val_accuracy: 0.8037
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4999 - accuracy: 0.8316 - val_loss: 0.5282 - val_accuracy:

In [27]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4590 - accuracy: 0.8401 - val_loss: 0.5109 - val_accuracy: 0.8075
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4509 - accuracy: 0.8429 - val_loss: 0.5048 - val_accuracy: 0.8072
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4427 - accuracy: 0.8413 - val_loss: 0.4965 - val_accuracy: 0.8086
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4299 - accuracy: 0.8437 - val_loss: 0.4532 - val_accuracy: 0.7908
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3809 - accuracy: 0.8336 - val_loss: 0.4428 - val_accuracy: 0.8042
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3675 - accuracy: 0.8428 - val_loss: 0.4224 - val_accuracy: 0.8039
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3575 - accuracy: 0.8476 - val_loss: 0.4169 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation